# Automatización Iber 3.3.1

Comenzamos con la modificación del modelo para el cambio automático de la condición de borde de entrada. Esto incluye la modificación de la CB, el remallado, la asignación de elevación para cada nodo

In [5]:
import pandas as pd

# Definir el directorio del proyecto (ajustar según sea necesario)
project_directory = "C:/temp/cruce_test.gid"

# Definir el archivo batch donde se guardarán los comandos
batch_file = "instructions.bch"

# Función para escribir en el archivo batch
def write_to_batch(commands):
    with open(batch_file, "w") as f:
        for command in commands:
            f.write(command + "\n")

# 1. Cargar el archivo del proyecto
load_project_command = f"MEscape Files Read {{{project_directory}}}"

# 2. Modificar la condición de borde (función de generación del comando)
def load_hydrograph(file_path="G:/Floods-WebMap/hydrograph.csv"):
    try:
        df = pd.read_csv(file_path, header=None, delim_whitespace=True)
        df.columns = ["Tiempo", "Caudal"]
        return df
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
        return None

def generate_boundary_condition_command(face_number, df):
    if df is None or df.empty:
        return "Error: No se han cargado datos del hidrograma."
    
    hydrograph_str = " ".join([f"{row['Tiempo']} {row['Caudal']}" for _, row in df.iterrows()])
    
    command = f"Mescape Data Conditions AssignCond 2D_Inlet Change 0 {{0 0 0}} Total_Discharge Critical/Subcritical #N# 8 {hydrograph_str} #N# 3 0.0 0.0 0.0 1 _Subcritical #N# 2 0.0 0.0 Elevation #N# 3 0.0 0.0 0.0 #N# 3 0.0 0.0 0.0 #N# 2 0.0 0.0 #N# 1 0.0 {face_number} escape"
    
    return command

# Leer el hidrograma y generar el comando
df_hydrograph = load_hydrograph()
boundary_condition_command = generate_boundary_condition_command(2, df_hydrograph)  # Número de cara por defecto: 2

# 3. Generar la malla
meshing_commands = [
    "Mescape Meshing Generate Yes {-tcl- GiD_RaiseEvent GiD_Event_BeforeMeshGeneration 0.551582} escape",
    "Mescape Utilities SwapNormals Surfaces SelByNormal 1:end escape 0.0 0.0 1.0 Yes escape",
    "Mescape Meshing Generate 0.551582 MeshingParametersFrom=Preferences escape 'SetViewMode Mesh"
]

# Escribir los comandos en el archivo batch
all_commands = [load_project_command, boundary_condition_command] + meshing_commands
write_to_batch(all_commands)

print(f"Archivo batch '{batch_file}' generado con éxito en el directorio actual.")


Archivo batch 'instructions.bch' generado con éxito en el directorio actual.


C:\Users\EPFL-LHE\AppData\Local\Temp\ipykernel_23432\94994252.py:21: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, header=None, delim_whitespace=True)


Una vez creado el proyecto, en vez de correr el caso desde la consola batch, preferimos llamar al solver de IberPlus directamente. Así podremos hacer uso de CUDA para acelerar el cálculo.

In [38]:
import os
import sys
import subprocess

# Ruta del ejecutable de Iber (ajústala según la instalación)
iber_exe = r'C:\Program Files\Iber\Iber 3.3.1\problemtypes\IBER.gid\bin\windows\IberPlus.exe'  

# Lista de modelos Iber a ejecutar (definida como lista por si se necesitan varios)
modelos_iber = [r'C:\temp\cruce_test.gid']  # Agregar más modelos si es necesario

print("\tEjecutando modelos Iber:")

for i, modelo in enumerate(modelos_iber, start=1):
    os.chdir(modelo)  # Cambia al directorio del modelo
    print(f"\r\tEjecutando {i}/{len(modelos_iber)}: {modelo}")

    # Llamar al ejecutable de Iber
    subprocess.call(iber_exe)

    print("\tModelo completado.")

print("\nTodos los modelos han sido ejecutados.")


	Ejecutando modelos Iber:
	Ejecutando 1/1: C:\temp\cruce_test.gid
	Modelo completado.

Todos los modelos han sido ejecutados.


In [44]:
import os
import subprocess

# Definir el archivo batch para el postproceso
postprocess_batch_file = "postprocess.bch"

# Crear el contenido del archivo batch para el postproceso
postprocess_commands = f"""MEscape Files Read {{{project_directory}}}
Mescape Postprocess escape
*****tcl set ::Reproject 0
*****tcl set ::Interpolation linear
*****tcl set ::cell 0.1
*****tcl set ::Timeresults Allsteps
*****tcl set ::PostRaster(thisanalysis) Hydraulic
*****tcl set ::PostRaster(thisstep) =
*****tcl set ::PostRaster(over) elements
*****tcl set ::PostRaster(thiszone) 1
*****tcl set ::PostRaster(thishemis) N
*****tcl set ::PostRaster(output,number) 1
*****tcl set "::PostRaster(output,Depth (m))" 1
*****tcl set ::PostRaster(output,Froude) 0
*****tcl set ::PostRaster(output,connectivities) 0
*****tcl set "::PostRaster(output,Water Elevation (m))" 0
*****tcl set "::PostRaster(output,Specific Discharge (m2/s))" 0
*****tcl set ::PostRaster(output,coordinates) 0
*****tcl set "::PostRaster(output,Velocity (m/s))" 1
*****tcl ::PostRaster::ApplyListEntities nada
MEscape Quit No
"""

# Guardar el archivo batch en la misma carpeta que el otro batch
with open(postprocess_batch_file, "w") as f:
    f.write(postprocess_commands)

print(f"Archivo batch '{postprocess_batch_file}' generado con éxito.")

# Ruta del ejecutable de Iber
iber_gui_exe = r"C:\Program Files\Iber\Iber 3.3.1\gid.exe"

# Comando para ejecutar el postproceso en Iber
command = [iber_gui_exe, "-b", os.path.abspath(postprocess_batch_file), "-n"]

# Ejecutar el postproceso
print("\nEjecutando el postproceso en Iber...")
subprocess.run(command)

print("\nPostproceso completado.")


Archivo batch 'postprocess.bch' generado con éxito.

Ejecutando el postproceso en Iber...

Postproceso completado.


In [45]:
import os
import re

# Directorio donde se guardan los archivos ASC
asc_directory = r"C:\temp\cruce_test.gid\Rasters\Hydraulic"

# Expresión regular para detectar archivos con formato Depth____X.X.asc o Depth____X.asc
pattern = re.compile(r"Depth____(\d+)(?:\.\d+)?\.asc")

# Renombrar archivos
for filename in os.listdir(asc_directory):
    match = pattern.match(filename)
    if match:
        original_number = int(float(match.group(1)))  # Convertir a entero eliminando decimales
        new_number = f"{original_number:06d}"  # Convertir a 6 cifras con ceros a la izquierda
        new_filename = f"Depth____{new_number}.asc"
        old_path = os.path.join(asc_directory, filename)
        new_path = os.path.join(asc_directory, new_filename)
        
        # Renombrar el archivo
        os.rename(old_path, new_path)
        print(f"Renombrado: {filename} → {new_filename}")

print("\nTodos los archivos han sido renombrados correctamente.")


Renombrado: Depth____0.0.asc → Depth____000000.asc
Renombrado: Depth____1.0.asc → Depth____000001.asc
Renombrado: Depth____10.0.asc → Depth____000010.asc
Renombrado: Depth____100.0.asc → Depth____000100.asc
Renombrado: Depth____101.0.asc → Depth____000101.asc
Renombrado: Depth____102.0.asc → Depth____000102.asc
Renombrado: Depth____103.0.asc → Depth____000103.asc
Renombrado: Depth____104.0.asc → Depth____000104.asc
Renombrado: Depth____105.0.asc → Depth____000105.asc
Renombrado: Depth____106.0.asc → Depth____000106.asc
Renombrado: Depth____107.0.asc → Depth____000107.asc
Renombrado: Depth____108.0.asc → Depth____000108.asc
Renombrado: Depth____109.0.asc → Depth____000109.asc
Renombrado: Depth____11.0.asc → Depth____000011.asc
Renombrado: Depth____110.0.asc → Depth____000110.asc
Renombrado: Depth____111.0.asc → Depth____000111.asc
Renombrado: Depth____112.0.asc → Depth____000112.asc
Renombrado: Depth____113.0.asc → Depth____000113.asc
Renombrado: Depth____114.0.asc → Depth____000114.asc

In [46]:
import os
import re
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio.io import MemoryFile
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pyproj import Transformer
from PIL import Image, ImageDraw, ImageFont

# ====================================================
# PARÁMETROS GLOBALES
# ====================================================

INPUT_DIR = r"C:\temp\cruce_test.gid\Rasters\Hydraulic"  # Carpeta donde están los archivos ASC
OUTPUT_DIR = r"C:\temp\cruce_test.gid\Rasters\Hydraulic\Processed"  # Carpeta donde se guardarán los PNGs y TIFs
USER_MAX_DEPTH = 2.0  # Valor máximo de profundidad en metros
SRC_CRS = "EPSG:2056"  # CRS de origen (CH1903+ / LV95)
DST_CRS = "EPSG:3857"  # CRS de salida (Web Mercator)

# Crear la carpeta de salida si no existe
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# ====================================================
# FUNCIÓN PARA LEER ARCHIVOS ASC
# ====================================================

def read_ascii_xllcenter(filename):
    """Lee un archivo ASC y extrae sus datos y metadatos."""
    with open(filename, 'r') as f:
        header_lines = [next(f) for _ in range(6)]
    
    ncols = nrows = None
    xllcenter = yllcenter = None
    cellsize = None
    nodata_val = None

    for line in header_lines:
        parts = re.split(r"\s+", line.strip())
        key = parts[0].upper()
        val = float(parts[-1])
        if key.startswith("NCOLS"):
            ncols = int(val)
        elif key.startswith("NROWS"):
            nrows = int(val)
        elif key.startswith("XLLCENTER"):
            xllcenter = val
        elif key.startswith("YLLCENTER"):
            yllcenter = val
        elif key.startswith("CELLSIZE"):
            cellsize = val
        elif key.startswith("NODATA_VALUE"):
            nodata_val = val

    data = np.loadtxt(filename, skiprows=6)
    x_min = xllcenter - 0.5 * cellsize
    y_max = yllcenter + (nrows - 0.5) * cellsize
    transform = from_origin(x_min, y_max, cellsize, cellsize)
    return data, transform, ncols, nrows, nodata_val

# ====================================================
# FUNCIÓN PARA CONVERTIR ASC A PNG
# ====================================================

def asc_to_png(filename_asc, output_png):
    """Convierte un archivo ASC en una imagen PNG con una rampa de colores."""
    data, transform, width, height, nodata_val = read_ascii_xllcenter(filename_asc)
    
    low_rgb = np.array([173, 216, 230], dtype=np.float32)  # Azul claro
    high_rgb = np.array([0, 0, 139], dtype=np.float32)      # Azul oscuro
    limit_val = 0.9 * USER_MAX_DEPTH

    band = data
    valid_mask = band != nodata_val
    scaled = ((band - 0) / (limit_val - 0)) * 255.0
    scaled = np.clip(scaled, 0, 255).astype(np.float32)

    R = low_rgb[0] + (high_rgb[0] - low_rgb[0]) * (scaled / 255.0)
    G = low_rgb[1] + (high_rgb[1] - low_rgb[1]) * (scaled / 255.0)
    B = low_rgb[2] + (high_rgb[2] - low_rgb[2]) * (scaled / 255.0)

    R = np.clip(R, 0, 255).astype(np.uint8)
    G = np.clip(G, 0, 255).astype(np.uint8)
    B = np.clip(B, 0, 255).astype(np.uint8)
    A = np.where(valid_mask, 255, 0).astype(np.uint8)

    img = np.dstack([R, G, B, A])
    Image.fromarray(img).save(output_png)

# ====================================================
# FUNCIÓN PARA RENOMBRAR LOS ARCHIVOS ASC
# ====================================================

def format_filename(filename):
    """Renombra archivos ASC a PNG con 6 cifras en el nombre."""
    match = re.search(r"Depth____(\d+)\.asc", filename)
    if match:
        num = int(match.group(1))
        formatted_num = f"{num:06d}"
        return f"Depth_{formatted_num}.png"
    return None

def extract_number(filename):
    """Extrae el número de tiempo de los archivos ASC."""
    match = re.search(r"Depth____(\d+)\.asc", filename)
    return int(match.group(1)) if match else float('inf')

# ====================================================
# PROCESAMIENTO DE ARCHIVOS
# ====================================================

# Listar archivos ASC y ordenarlos por tiempo
asc_files = [f for f in os.listdir(INPUT_DIR) if f.startswith("Depth____") and f.endswith(".asc")]
asc_files = sorted(asc_files, key=extract_number)

print(f"Se encontraron {len(asc_files)} archivos ASC para procesar.")

for filename in asc_files:
    input_path = os.path.join(INPUT_DIR, filename)
    output_name = format_filename(filename)  # Ejemplo: Depth_000123.png

    if output_name is not None:
        output_path = os.path.join(OUTPUT_DIR, output_name)
        asc_to_png(input_path, output_path)
        print(f"{filename} → {output_name}")

print("\nProceso completado. Archivos PNG generados en:", OUTPUT_DIR)


Se encontraron 351 archivos ASC para procesar.
Depth____000000.asc → Depth_000000.png
Depth____000001.asc → Depth_000001.png
Depth____000002.asc → Depth_000002.png
Depth____000003.asc → Depth_000003.png
Depth____000004.asc → Depth_000004.png
Depth____000005.asc → Depth_000005.png
Depth____000006.asc → Depth_000006.png
Depth____000007.asc → Depth_000007.png
Depth____000008.asc → Depth_000008.png
Depth____000009.asc → Depth_000009.png
Depth____000010.asc → Depth_000010.png
Depth____000011.asc → Depth_000011.png
Depth____000012.asc → Depth_000012.png
Depth____000013.asc → Depth_000013.png
Depth____000014.asc → Depth_000014.png
Depth____000015.asc → Depth_000015.png
Depth____000016.asc → Depth_000016.png
Depth____000017.asc → Depth_000017.png
Depth____000018.asc → Depth_000018.png
Depth____000019.asc → Depth_000019.png
Depth____000020.asc → Depth_000020.png
Depth____000021.asc → Depth_000021.png
Depth____000022.asc → Depth_000022.png
Depth____000023.asc → Depth_000023.png
Depth____000024.a

Si no está instalado Git:

```raw
!conda install -c conda-forge git -y


!git --version


!git config --global user.name "cgotelli"
!git config --global user.email "clemente.gotelli@outlook.com"


!git config --global --list
```

Una vez listo eso, clonamos:

```raw
!git clone https://github.com/cgotelli/Floods-WebMap.git G:/Floods-WebMap
```

In [52]:
import shutil
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont

# Directorios en el disco G:
local_repo_dir = r"G:\Floods-WebMap"  # Directorio del repo clonado en G:
source_images_dir = r"C:\temp\cruce_test.gid\Rasters\Hydraulic\Processed"  # PNGs generados
source_asc_dir = r"C:\temp\cruce_test.gid\Rasters\Hydraulic"  # Archivos ASC generados

# Directorios de destino en el repositorio en G:
target_png_dir = os.path.join(local_repo_dir, "simulation_files", "png")
target_asc_dir = os.path.join(local_repo_dir, "simulation_files", "asc")
target_aux_dir = os.path.join(local_repo_dir, "aux_images")  # Carpeta para la leyenda

# Parámetros de la leyenda
LEGEND_WIDTH = 50
LEGEND_HEIGHT = 256
TOP_MARGIN = 40
BOTTOM_MARGIN = 20
LEFT_MARGIN = 25
RIGHT_MARGIN = 10
USER_MAX_DEPTH = 2.0  # Máxima profundidad a considerar en la leyenda

# Función para limpiar carpetas antes de copiar nuevos archivos (excepto aux_images)
def clean_directory(directory):
    if os.path.exists(directory):
        for file in os.listdir(directory):
            file_path = os.path.join(directory, file)
            try:
                os.remove(file_path)
                print(f"🗑️ Eliminado: {file}")
            except Exception as e:
                print(f"⚠️ Error al eliminar {file}: {e}")
    else:
        os.makedirs(directory)

# Limpiar carpetas PNG y ASC antes de copiar nuevos archivos (NO TOCAR aux_images)
print("🧹 Eliminando archivos anteriores en G:/Floods-WebMap/simulation_files/png...")
clean_directory(target_png_dir)

print("🧹 Eliminando archivos anteriores en G:/Floods-WebMap/simulation_files/asc...")
clean_directory(target_asc_dir)

# Función para copiar archivos
def copy_files(source_dir, target_dir, extension):
    for file in os.listdir(source_dir):
        if file.endswith(extension):
            shutil.copy2(os.path.join(source_dir, file), os.path.join(target_dir, file))
            print(f"📂 Copiado: {file}")

# Copiar nuevos archivos PNG
print("\n📤 Copiando nuevos archivos PNG...")
copy_files(source_images_dir, target_png_dir, ".png")

# Copiar nuevos archivos ASC
print("\n📤 Copiando nuevos archivos ASC...")
copy_files(source_asc_dir, target_asc_dir, ".asc")

# ====================================================
# Generar la imagen de la leyenda (SIN ELIMINAR ARCHIVOS ANTERIORES EN aux_images)
# ====================================================

print("\n🎨 Generando la leyenda...")

# Crear imagen RGBA para la leyenda
legend_array = np.ones((LEGEND_HEIGHT, LEGEND_WIDTH, 4), dtype=np.uint8) * 255
legend_array[..., 3] = 128  # Fondo con 50% de opacidad

# Gradiente de color (azul claro a azul oscuro)
low_rgb = np.array([173, 216, 230], dtype=np.float32)  # Azul claro
high_rgb = np.array([0, 0, 139], dtype=np.float32)  # Azul oscuro
limit_val = 0.9 * USER_MAX_DEPTH

inner_height = LEGEND_HEIGHT - TOP_MARGIN - BOTTOM_MARGIN
for y in range(inner_height):
    factor = y / float(inner_height - 1)
    inv_factor = 1 - factor  # Invertir para que la parte superior sea azul oscuro
    color = low_rgb + (high_rgb - low_rgb) * inv_factor
    color = np.clip(color, 0, 255).astype(np.uint8)
    for x in range(LEFT_MARGIN, LEGEND_WIDTH - RIGHT_MARGIN):
        legend_array[TOP_MARGIN + y, x, :3] = color
        legend_array[TOP_MARGIN + y, x, 3] = 255  # Opaque en la barra de color

# Convertir a imagen
legend_img = Image.fromarray(legend_array, mode="RGBA")
draw = ImageDraw.Draw(legend_img)
font = ImageFont.load_default()

# Etiquetas
title_text = "Depth [m]"
draw.text(((LEGEND_WIDTH - draw.textlength(title_text, font)) // 2, 2), title_text, font=font, fill=(0, 0, 0, 255))

# Min label (0)
draw.text((LEFT_MARGIN - 10, LEGEND_HEIGHT - BOTTOM_MARGIN), "0", font=font, fill=(0, 0, 0, 255))

# Max label (90% de USER_MAX_DEPTH)
max_label = f"{limit_val:.1f}"
draw.text((LEFT_MARGIN - 10, TOP_MARGIN - 10), max_label, font=font, fill=(0, 0, 0, 255))

# Guardar la leyenda en G:\Floods-WebMap\aux_images (NO SE ELIMINAN ARCHIVOS ANTERIORES)
legend_output = os.path.join(target_aux_dir, "legend.png")
legend_img.save(legend_output)
print(f"✅ Leyenda guardada en {legend_output}")

print("\n✅ Todos los archivos han sido copiados en G:/Floods-WebMap.")

🧹 Eliminando archivos anteriores en G:/Floods-WebMap/simulation_files/png...
🗑️ Eliminado: Depth_000000.png
🗑️ Eliminado: Depth_000001.png
🗑️ Eliminado: Depth_000002.png
🗑️ Eliminado: Depth_000003.png
🗑️ Eliminado: Depth_000004.png
🗑️ Eliminado: Depth_000005.png
🗑️ Eliminado: Depth_000006.png
🗑️ Eliminado: Depth_000007.png
🗑️ Eliminado: Depth_000008.png
🗑️ Eliminado: Depth_000009.png
🗑️ Eliminado: Depth_000010.png
🗑️ Eliminado: Depth_000011.png
🗑️ Eliminado: Depth_000012.png
🗑️ Eliminado: Depth_000013.png
🗑️ Eliminado: Depth_000014.png
🗑️ Eliminado: Depth_000015.png
🗑️ Eliminado: Depth_000016.png
🗑️ Eliminado: Depth_000017.png
🗑️ Eliminado: Depth_000018.png
🗑️ Eliminado: Depth_000019.png
🗑️ Eliminado: Depth_000020.png
🗑️ Eliminado: Depth_000021.png
🗑️ Eliminado: Depth_000022.png
🗑️ Eliminado: Depth_000023.png
🗑️ Eliminado: Depth_000024.png
🗑️ Eliminado: Depth_000025.png
🗑️ Eliminado: Depth_000026.png
🗑️ Eliminado: Depth_000027.png
🗑️ Eliminado: Depth_000028.png
🗑️ Eliminado: Depth_0000

In [53]:
cd G:/Floods-WebMap


G:\Floods-WebMap


In [54]:
!git add simulation_files/png
!git add simulation_files/asc
!git add aux_images/legend.png
!git commit -m "Actualización de imágenes PNG, archivos ASC y leyenda en aux_images (sin eliminar previos)"
!git push origin main


[main e792c06] ActualizaciÃ³n de imÃ¡genes PNG, archivos ASC y leyenda en aux_images (sin eliminar previos)
 2088 files changed, 120744 insertions(+), 341600 deletions(-)
 delete mode 100644 simulation_files/asc/.DS_Store
 delete mode 100755 simulation_files/asc/Depth____0.0.asc
 create mode 100644 simulation_files/asc/Depth____000000.asc
 create mode 100644 simulation_files/asc/Depth____000001.asc
 create mode 100644 simulation_files/asc/Depth____000002.asc
 create mode 100644 simulation_files/asc/Depth____000003.asc
 create mode 100644 simulation_files/asc/Depth____000004.asc
 create mode 100644 simulation_files/asc/Depth____000005.asc
 create mode 100644 simulation_files/asc/Depth____000006.asc
 create mode 100644 simulation_files/asc/Depth____000007.asc
 create mode 100644 simulation_files/asc/Depth____000008.asc
 create mode 100644 simulation_files/asc/Depth____000009.asc
 create mode 100644 simulation_files/asc/Depth____000010.asc
 create mode 100644 simulation_files/asc/Depth___

To https://github.com/cgotelli/Floods-WebMap.git
   42fdb2b..e792c06  main -> main


In [58]:
# ========================================
# Example snippet for build_map.ipynb
# Updated to include two animation layers and allow customization of the HTML title and layer names.
# Only Animation Layer 1 is active by default.
# ========================================

output_html_filename = "my_map.html"

# User-defined variables for customization:
html_title = "My Custom Map Title"           # Title of the HTML page
animation_layer_name = "Animation Layer 1"    # Name for the first animation layer
animation_layer2_name = "Animation Layer 2"   # Name for the second animation layer
drawings_layer_name = "Drawings Layer"        # Name for the drawings layer

# Define the GitHub folder URL (the one seen in the browser)
github_url = "https://github.com/cgotelli/Floods-WebMap/tree/main/simulation_files/png"

# Legend image location in the new repo:
legend_url = "https://github.com/cgotelli/Floods-WebMap/raw/refs/heads/main/aux_images/legend.png"

default_zoom = 15
overlay_opacity = 0.75
start_year = 2025
start_month = 1
start_day = 1

# Convert the GitHub URL to the corresponding API URL.
# GitHub URL format:
#   https://github.com/{user}/{repo}/tree/{branch}/{path}
# API URL format:
#   https://api.github.com/repos/{user}/{repo}/contents/{path}
if "github.com" in github_url:
    # Remove the "https://github.com/" prefix.
    path_part = github_url.replace("https://github.com/", "")
    # Split the remaining part.
    parts = path_part.split("/")
    # Expecting parts to be: [user, repo, "tree", branch, path...]
    if len(parts) >= 5 and parts[2] == "tree":
        user = parts[0]
        repo = parts[1]
        branch = parts[3]  # branch is not used in API URL
        folder_path = "/".join(parts[4:])
        folder_url = f"https://api.github.com/repos/{user}/{repo}/contents/{folder_path}"
    else:
        # Fallback to the provided URL if the format is unexpected
        folder_url = github_url
else:
    folder_url = github_url

# Suppose we've computed bounding_boxes from the ASC->PNG script.
# We'll use the first bounding box for demonstration:
if 'bounding_boxes' in globals() and bounding_boxes:
    # bounding_boxes[i] => (output_png_name, (lat_bottom, lon_left, lat_top, lon_right))
    _, (lat_bottom, lon_left, lat_top, lon_right) = bounding_boxes[0]
else:
    # Fallback or default values
    lat_bottom, lon_left, lat_top, lon_right = 46.5199, 6.6309, 46.5211, 6.6363

# Build the JavaScript strings for the Leaflet bounds and center.
imageBounds_str = f"[[{lat_bottom}, {lon_left}], [{lat_top}, {lon_right}]]"
center_lat = 0.5 * (lat_bottom + lat_top)
center_lon = 0.5 * (lon_left + lon_right)
mapCenter_str = f"[{center_lat:.5f}, {center_lon:.5f}]"

# Start date for frames (JS-compatible)
startDateJS = f"new Date('{start_year:04d}-{start_month:02d}-{start_day:02d}T00:00:00Z')"

# Build the HTML content with all text in English and customized title and layer names.
html_content = f"""<!DOCTYPE html>
<html>
<head>
  <title>{html_title}</title>
  <meta charset="utf-8"/>
  <!-- CSS: Leaflet -->
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.5.1/dist/leaflet.css" />
  <!-- CSS: Leaflet TimeDimension -->
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet-timedimension@1.1.1/dist/leaflet.timedimension.control.min.css" />
  <!-- CSS: Leaflet Measure -->
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet-measure@3.1.0/dist/leaflet-measure.min.css">
  <!-- CSS: Leaflet.draw (v0.4.2) -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/leaflet.draw/0.4.2/leaflet.draw.css"/>
  <!-- CSS: FontAwesome for icons -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
  <style>
    #map {{
      position: relative;
      width: 100%;
      height: 600px;
    }}
  </style>
</head>
<body>
  <div id="map">
    <!-- Legend in the lower-right corner -->
    <div id="legend" style="position: absolute; bottom: 15px; right: 10px; background: rgba(255,255,255,0); padding: 5px; border: 0px solid #ccc; z-index: 1000;">
      <img src="{legend_url}" alt="Depth Legend">
    </div>
  </div>
  
  <!-- JS: Leaflet, Iso8601, TimeDimension, Measure, Leaflet.draw -->
  <script src="https://cdn.jsdelivr.net/npm/leaflet@1.5.1/dist/leaflet.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/iso8601-js-period@0.2.1/iso8601.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/leaflet-timedimension@1.1.1/dist/leaflet.timedimension.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/leaflet-measure@3.1.0/dist/leaflet-measure.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet.draw/0.4.2/leaflet.draw.js"></script>
  
  <script>
    /*************************************************
     * preloadImages: loads all image URLs in parallel
     *************************************************/
    function preloadImages(urls, callback) {{
      let loadedCount = 0;
      const total = urls.length;
      urls.forEach(url => {{
        const img = new Image();
        img.onload = img.onerror = function() {{
          loadedCount++;
          if (loadedCount === total) {{
            callback(urls);
          }}
        }};
        img.src = url;
      }});
    }}
    
    /*************************************************
     * getPNGFilesFromGitHubFolder: fetch Depth_*.png from new repo
     *************************************************/
    function getPNGFilesFromGitHubFolder(folderUrl, callback) {{
      fetch(folderUrl)
        .then(response => response.json())
        .then(json => {{
          let pngFiles = json.filter(item => 
            item.type === "file" && item.name.toLowerCase().startsWith("depth")
          ).map(item => item.download_url);
          pngFiles.sort();
          callback(pngFiles);
        }})
        .catch(error => {{
          console.error("Error fetching GitHub folder:", error);
          callback([]);
        }});
    }}
    
    /*************************************************
     * Subclass for ImageOverlay in TimeDimension
     *************************************************/
    L.TimeDimension.Layer.ImageOverlay = L.TimeDimension.Layer.extend({{
      initialize: function(layer, options) {{
        L.TimeDimension.Layer.prototype.initialize.call(this, layer, options);
        if (typeof this.options.time === 'string') {{
          this._time = new Date(this.options.time).getTime();
        }} else if (this.options.time instanceof Date) {{
          this._time = this.options.time.getTime();
        }} else {{
          this._time = this.options.time;
        }}
      }},
      _onNewTimeLoading: function(ev) {{
        this.fire('timeload', {{ time: ev.time }});
      }},
      isReady: function(time) {{
        return true;
      }},
      _update: function() {{
        if (!this._map) return;
        var currentTime = this._timeDimension.getCurrentTime();
        var tolerance = 500;
        console.log("Overlay time: " + this._time + ", currentTime: " + currentTime);
        if (Math.abs(currentTime - this._time) < tolerance) {{
          if (!this._map.hasLayer(this._baseLayer)) {{
            console.log("Adding overlay for time " + this._time);
            this._map.addLayer(this._baseLayer);
          }}
        }} else {{
          if (this._map.hasLayer(this._baseLayer)) {{
            console.log("Removing overlay for time " + this._time);
            this._map.removeLayer(this._baseLayer);
          }}
        }}
        return true;
      }}
    }});
    L.timeDimension.layer.imageOverlay = function(layer, options) {{
      return new L.TimeDimension.Layer.ImageOverlay(layer, options);
    }};
    
    /*************************************************
     * Initialization
     *************************************************/
    var folderUrl = "{folder_url}";
    
    getPNGFilesFromGitHubFolder(folderUrl, function(pngFiles) {{
      console.log("Obtained PNG URLs:", pngFiles);
      
      preloadImages(pngFiles, function(loadedUrls) {{
        console.log("All images have been preloaded.");
        
        var times = [];
        var startTime = {startDateJS};
        for (var i = 0; i < loadedUrls.length; i++) {{
          var time = new Date(startTime.getTime() + i * 1000);
          times.push(time.toISOString());
        }}
        
        var imageBounds = {imageBounds_str};
        var mapCenter = {mapCenter_str};
        
        var mapStartTime = times[0];
        var mapEndTime = times[times.length - 1];
        
        var map = L.map('map', {{
          center: mapCenter,
          zoom: {default_zoom},
          timeDimension: true,
          timeDimensionOptions: {{
            timeInterval: mapStartTime + "/" + mapEndTime,
            period: "PT1S",
            currentTime: new Date(mapStartTime).getTime()
          }}
        }});
        
        L.control.timeDimension({{
          autoPlay: true,
          loopButton: true,
          timeSliderDragUpdate: true,
          speedSlider: true,
          playerOptions: {{
            transitionTime: 100,
            loop: true,
            startOver: true
          }},
          minSpeed: 0.1,
          maxSpeed: 20,
          displayDate: true
        }}).addTo(map);
        
        /********************
         * Base layers
         ********************/
        var osmStandard = L.tileLayer('https://{{s}}.tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png', {{ 
          attribution: '© OpenStreetMap Contributors' 
        }});
        var swissColor = L.tileLayer(
          'https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.pixelkarte-farbe/default/current/3857/{{z}}/{{x}}/{{y}}.jpeg',
          {{ 
            attribution: '© swisstopo <img src="https://upload.wikimedia.org/wikipedia/commons/f/f3/Flag_of_Switzerland.svg" style="height:16px; vertical-align:middle;" alt="Swiss Flag">' 
          }}
        );
        var swissTopo = L.tileLayer(
          'https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.swissimage/default/current/3857/{{z}}/{{x}}/{{y}}.jpeg',
          {{ 
            attribution: '© swisstopo <img src="https://upload.wikimedia.org/wikipedia/commons/f/f3/Flag_of_Switzerland.svg" style="height:16px; vertical-align:middle;" alt="Swiss Flag">' 
          }}
        );
        var cartoPositron = L.tileLayer(
          'https://{{s}}.basemaps.cartocdn.com/light_all/{{z}}/{{x}}/{{y}}{{r}}.png',
          {{ attribution: '© OpenStreetMap contributors, © CartoDB' }}
        );
        var cartoDark = L.tileLayer(
          'https://{{s}}.basemaps.cartocdn.com/dark_all/{{z}}/{{x}}/{{y}}{{r}}.png',
          {{ attribution: '© OpenStreetMap contributors, © CartoDB' }}
        );
        
        var baseLayers = {{
          "OSM Standard": osmStandard,
          "Swiss Color": swissColor,
          "Swiss Topo": swissTopo,
          "CartoDB Positron": cartoPositron,
          "CartoDB Dark Matter": cartoDark
        }};
        
        /********************
         * Overlay layers
         ********************/
        // FeatureGroup for drawings (Leaflet.draw) - now added by default
        var drawnItems = new L.FeatureGroup();
        drawnItems.addTo(map);
        
        // Animation layer 1: group all time-based image overlays
        var animationLayer = L.layerGroup();
        for (let i = 0; i < loadedUrls.length; i++) {{
          var overlay = L.imageOverlay(loadedUrls[i], imageBounds, {{ opacity: {overlay_opacity} }});
          var tdLayer = L.timeDimension.layer.imageOverlay(overlay, {{ time: times[i] }});
          animationLayer.addLayer(tdLayer);
        }}
        animationLayer.addTo(map);
        
        // Animation layer 2: identical to the first (for demonstration)
        var animationLayer2 = L.layerGroup();
        for (let i = 0; i < loadedUrls.length; i++) {{
          var overlay = L.imageOverlay(loadedUrls[i], imageBounds, {{ opacity: {overlay_opacity} }});
          var tdLayer = L.timeDimension.layer.imageOverlay(overlay, {{ time: times[i] }});
          animationLayer2.addLayer(tdLayer);
        }}
        
        // Layers control: base layers and overlays (with user-defined names)
        L.control.layers(baseLayers, {{
          "{animation_layer_name}": animationLayer,
          "{animation_layer2_name}": animationLayer2,
          "{drawings_layer_name}": drawnItems
        }}).addTo(map);
        
        /********************
         * Measure control
         ********************/
        var measureControl = new L.Control.Measure({{
          primaryLengthUnit: 'meters',
          primaryAreaUnit: 'hectares',
          position: 'topleft'
        }});
        measureControl.addTo(map);
        
        // Add one of the base layers by default
        swissTopo.addTo(map);
        
        /*************************************************
         * Leaflet.draw
         *************************************************/
        var drawControl = new L.Control.Draw({{
          edit: {{ featureGroup: drawnItems }},
          draw: {{
            polygon: true,
            polyline: true,
            rectangle: true,
            circle: true,
            marker: true
          }}
        }});
        map.addControl(drawControl);
        
        // --- New Control: Button to save drawings ---
        var saveDrawingsControl = L.Control.extend({{
          options: {{
            position: 'topleft'
          }},
          onAdd: function(map) {{
            var container = L.DomUtil.create('div', 'leaflet-bar leaflet-control');
            container.style.backgroundColor = 'white';
            container.style.padding = '10px';
            container.style.cursor = 'pointer';
            container.title = 'Save drawings';
            container.innerHTML = '<i class="fa fa-save"></i>';
            L.DomEvent.disableClickPropagation(container);
            L.DomEvent.on(container, 'click', function(e) {{
              // Convert the drawings layer to GeoJSON
              var data = drawnItems.toGeoJSON();
              var convertedData = JSON.stringify(data, null, 2);
              // Create a Blob and generate a URL for download
              var blob = new Blob([convertedData], {{ type: 'application/json' }});
              var url = URL.createObjectURL(blob);
              var a = document.createElement('a');
              a.href = url;
              a.download = 'drawings.geojson';
              document.body.appendChild(a);
              a.click();
              document.body.removeChild(a);
              URL.revokeObjectURL(url);
            }});
            return container;
          }}
        }});
        map.addControl(new saveDrawingsControl());
        // --- End of save control ---
        
        map.on('draw:created', function(e) {{
          var type = e.layerType,
              layer = e.layer;
          drawnItems.addLayer(layer);
          var infoText = "";
          if (type === 'polygon' || type === 'rectangle') {{
            var latlngs = layer.getLatLngs();
            if (Array.isArray(latlngs[0])) {{ latlngs = latlngs[0]; }}
            var area = L.GeometryUtil.geodesicArea(latlngs);
            var perimeter = 0;
            for (var j = 0; j < latlngs.length; j++) {{
              var next = latlngs[(j + 1) % latlngs.length];
              perimeter += latlngs[j].distanceTo(next);
            }}
            infoText = "Area: " + area.toFixed(2) + " m², Perimeter: " + perimeter.toFixed(2) + " m";
          }} else if (type === 'circle') {{
            var r = layer.getRadius();
            var area = Math.PI * r * r;
            var circumference = 2 * Math.PI * r;
            infoText = "Circle Area: " + area.toFixed(2) + " m², Circumference: " + circumference.toFixed(2) + " m";
          }} else if (type === 'polyline') {{
            var latlngs = layer.getLatLngs();
            var length = 0;
            for (var k = 0; k < latlngs.length - 1; k++) {{
              length += latlngs[k].distanceTo(latlngs[k+1]);
            }}
            infoText = "Length: " + length.toFixed(2) + " m";
          }} else if (type === 'marker') {{
            var latlng = layer.getLatLng();
            infoText = "Coordinates: " + latlng.lat.toFixed(5) + ", " + latlng.lng.toFixed(5);
          }}
          layer.bindPopup(infoText).openPopup();
        }});
      }});
    }});
  </script>
</body>
</html>
"""

# Finally, write the html_content to a file
with open(output_html_filename, "w", encoding="utf-8") as f:
    f.write(html_content)

print(f"HTML updated and exported to '{output_html_filename}'.")


HTML updated and exported to 'my_map.html'.


Original aquí abajo

```python
# -*- coding: utf-8 -*-
"""
Código para ejecutar varios modelos de Iber en serie

Gonzalo García-Alén Julio/2023
"""

import os
import sys
import subprocess

# Aquí indicamos la ruta de los modelos que queremos que sean ejecutados
ModelosIber = [r'E:\te7.gid', r'E:\te8.gid']

# Especificamos el ejecutable a usar, que ya debe estar en cada carpeta .gid
# Todos los ejecutables de Iber están en esta ruta dentro de la carpeta donde
# se ha instalado el programa: ...\Iber 3.3\problemtypes\IBER.gid\bin\windows
iber_exe = r'C:\Program Files\Iber\Iber 3.3.1\problemtypes\IBER.gid\bin\windows\IberPlus.exe'  # 'Iber.exe', 'IberPlus.exe' o 'RIber.exe'


print('\tEjecutando modelos Iber:')
for i in range(len(ModelosIber)):
    os.chdir(ModelosIber[i])
    a = ('\t' + str(i+1) + '/' + str(len(ModelosIber)))
    sys.stdout.write('\r'+a)
    subprocess.call( iber_exe )
    if i == (len(ModelosIber)-1): print('\n')
    print('done')
```